## Imports

In [11]:
from scipy.optimize import minimize
from itertools import combinations
import random

## Normalize the pool dictionaries

In [5]:
def Pnorm(pool):
    total = sum([1 / s[0] for s in pool.values()])
    for team in pool.keys():
        pool[team][0] = (1 / pool[team][0]) / total
    return pool

In [6]:
PoolA = Pnorm({'Ireland':[2246,4.7,0], 'Scotland':[2129,24,0], 'Japan':[2050,60,0], 'Russia':[1789,200,0], 'Samoa':[1965,200,0]})
PoolB = Pnorm({'New Zealand':[2323,1.76,0], 'South Africa':[2201,2.88,0], 'Italy':[1889,100,0], 'Namibia':[1838,200,0], 'Canada':[1839,100,0]})
PoolC = Pnorm({'England':[1.38,3.5,0], 'France':[5.6,12.5,0], 'Argentina':[7.2,20,0], 'Tonga':[500,100,0], 'USA':[500,100,0]})
PoolD = Pnorm({'Wales':[1.86,4.6,0], 'Australia':[2.12,6.8,0], 'Fiji':[40,60,0], 'Georgia':[500,100,0], 'Uruguay':[500,200,0]})

## ELO model

In [8]:
def expected_score(p1_rating, p2_rating):
    return 1 / (1 + 10**((p2_rating - p1_rating)/400))

In [7]:
expected_score(2000, 2000)

0.5

## Score function (How well the ELOs represent the odds)

In [61]:
def score(elos, odds, iterations=1000):
    players = len(elos)
    pool_wins = [0 for _ in range(players)]
    for _ in range(iterations):
        points = [0 for _ in range(players)]
        for i, j in combinations(range(players), 2):
            result = expected_score(elos[i], elos[j])
            if random.random() < result:
                points[i] += 1
            else:
                points[j] += 1
        best_score = max(points)
        pool_winner = random.choice([i for i in range(players) if points[i] == best_score])
        pool_wins[pool_winner] += 1.0/iterations
    
    score = sum(abs(a-b) for a,b in zip(pool_wins, odds))

    return score

In [62]:
score([2000, 2000, 2500], [0.1, 0.1, 0.8])

0.2280000000000006

## Optimization

In [ ]:
for pool in [PoolA, PoolB, PoolC, PoolD]:
    odds = [x[0] for x in pool.values()]
    elos = minimize(score, [2000 for _ in range(5)], args=odds, method='Nelder-Mead').x
    
    rounded_odds = sorted([round(o, 3) for o in odds])
    rounded_elos = sorted([int(e) for e in elos])
    
    print("ELOS:", *rounded_elos, sep=" ")
    print("Odds:", *rounded_odds, sep=" ", end="\n\n")

ELOS: 1994 2005 2007 2009 2024
Odds: 0.18 0.19 0.197 0.206 0.226

